# Function Calling/Tool Use


## Just a quick definition

Function calling in the context of Large Language Models (LLMs) refers to the ability of the model to invoke external functions or tools during its response generation. Instead of only generating text, the LLM can recognize when a task requires external data or computation, call a predefined function with the appropriate arguments, and then use the function's output to continue its response.

Think of having a super smart robot that you can only speak with. Pretty useless at doing anything aside from talking. Now imagine you have given that robot a hammer and some nails. It can now put up that wall painting that's been waiting forever to be hung :D

Except, the way we as AI Engineers give AI tools is by defining python functions and describing that function in detail to the LLM so it knows what tools it has access to, what each tool can do, what are its inputs and expected outputs.

Just remember, a *tool* in its simplest form is just a *python function* that you have defined. It can be as simple as a calculator function or something like being able to call external APIs.

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    raise Exception("API key is missing")

In [2]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [3]:
get_weather(24.343627, 54.497922)

39.6

In [4]:
import openai

client = openai.Client()

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the weather like in Abu Dhabi today? Reply only with the temperature in Celsius."},
    ]
)

print(response.output_text)

I'm unable to provide real-time data, including current weather conditions. You can check a weather website or app for the latest temperature in Abu Dhabi.


In [5]:
response.__dict__

{'id': 'resp_689304e149888191a8881d8d9596649a0ac8922b976163d5',
 'created_at': 1754465505.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_689304e232048191b4c0fd99f74d6bf10ac8922b976163d5', content=[ResponseOutputText(annotations=[], text="I'm unable to provide real-time data, including current weather conditions. You can check a weather website or app for the latest temperature in Abu Dhabi.", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [],
 'top_p': 1.0,
 'background': False,
 'max_output_tokens': None,
 'max_tool_calls': None,
 'previous_response_id': None,
 'prompt': None,
 'prompt_cache_key': None,
 'reasoning': Reasoning(effort=None, generate_summary=None, summary=None),
 'safety_identifier': None,
 'service_tier

Lets try giving our `get_weather` function as a tool to our AI.

Step 1: Define the tool in a format OpenAI can understand

In [6]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

Step 2: Pass the json over to the model

In [7]:
from pprint import pprint

input_messages = [{"role": "user", "content": "What's the weather like in Abu Dhabi today?"}]

response = client.responses.create(
    model="gpt-4o-mini",
    input=input_messages,
    tools=tools,
)

pprint(response)

Response(id='resp_6893051619cc819cadecb589f57901340625d1383c6d24a2', created_at=1754465558.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFunctionToolCall(arguments='{"latitude":24.4539,"longitude":54.3872}', call_id='call_BImXqbKiHcowxtu3KfLaaVtB', name='get_weather', type='function_call', id='fc_68930516f414819c97bab8e4bcc2a5a80625d1383c6d24a2', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'latitude': {'type': 'number'}, 'longitude': {'type': 'number'}}, 'required': ['latitude', 'longitude'], 'additionalProperties': False}, strict=True, type='function', description='Get current temperature for provided coordinates in celsius.')], top_p=1.0, background=False, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reason

In [8]:
pprint(response.__dict__)

{'_request_id': 'req_52be7bd56bdc116bf534ff5a5ce205c3',
 'background': False,
 'created_at': 1754465558.0,
 'error': None,
 'id': 'resp_6893051619cc819cadecb589f57901340625d1383c6d24a2',
 'incomplete_details': None,
 'instructions': None,
 'max_output_tokens': None,
 'max_tool_calls': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [ResponseFunctionToolCall(arguments='{"latitude":24.4539,"longitude":54.3872}', call_id='call_BImXqbKiHcowxtu3KfLaaVtB', name='get_weather', type='function_call', id='fc_68930516f414819c97bab8e4bcc2a5a80625d1383c6d24a2', status='completed')],
 'parallel_tool_calls': True,
 'previous_response_id': None,
 'prompt': None,
 'prompt_cache_key': None,
 'reasoning': Reasoning(effort=None, generate_summary=None, summary=None),
 'safety_identifier': None,
 'service_tier': 'default',
 'status': 'completed',
 'temperature': 1.0,
 'text': ResponseTextConfig(format=ResponseFormatText(type='text')),
 'tool_choice': 'auto',
 'to

In [9]:
pprint(response.output[0].__dict__)

{'arguments': '{"latitude":24.4539,"longitude":54.3872}',
 'call_id': 'call_BImXqbKiHcowxtu3KfLaaVtB',
 'id': 'fc_68930516f414819c97bab8e4bcc2a5a80625d1383c6d24a2',
 'name': 'get_weather',
 'status': 'completed',
 'type': 'function_call'}


In [ ]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

print("Tool call: ", tool_call)
print("Arguments: ", args)

In [ ]:
tool_call.arguments

We now need to pass on the arguments received by the model to our python function or tool

In [ ]:
result = get_weather(args["latitude"], args["longitude"])
result

In [ ]:
input_messages.append(tool_call)  # append model's function call message
input_messages.append({                               # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

pprint(input_messages)

In [ ]:
response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)
pprint(response_2.output_text)

# Resources:

- [OpenAIs Function Calling Guide](https://platform.openai.com/docs/guides/function-calling?api-mode=responses)

## Your Turn: Build Your Own Custom Tool

Now that you've seen how to define a Python function as a tool and connect it to an LLM, it's time to get creative!

### Instructions
- Think of a real-world use case where an LLM could benefit from calling a custom function.
- Examples: currency conversion, fetching stock prices, summarizing text, generating random jokes, or anything you find interesting.
- Define your function in Python.
- Write a tool definition (JSON schema) for your function.
- Pass your tool to the LLM and test it with a relevant prompt.
- Print and inspect the results.

---

Try to:
- Use clear function names and docstrings.
- Handle input arguments and outputs carefully.
- Print the LLM's tool call and your function's output in a readable way.

Be sure to place your submissions in `AI-Engineering-Intermediate/Part1/community-contributions`

I'm super excited to see what you come up with :D